In [ ]:
import os
import glob
import json
import openpyxl
import pandas as pd
import pandas.io.json as pd_json

In [ ]:
def listar_arquivos(extensao='json'):
    search = "output/*.{}".format(extensao)
    files = glob.glob(search)
    files.sort()
    return files

In [ ]:
def ler_json(caminho):
    with open(caminho) as fp:  
        dados = json.load(fp)
    return dados

In [ ]:
def obter_os_contrato(nome_arquivo):
    nome_sem_extensao = nome_arquivo.split('.')[0]
    partes = nome_sem_extensao.split('_')
    return int(partes[1]), int(partes[3])

In [ ]:
def obter_codigo_nome_os():
    arquivo = 'input/contratos-controle.xlsx'
    df = pd.read_excel(arquivo, sheet_name='contratos')
    cod_os_nome = {}
    for os in df['ORGANIZACAO']:
        codigo_nome = os.split(' - ')
        cod_os_nome.update({int(codigo_nome[0]): codigo_nome[1]})
    return cod_os_nome

In [ ]:
def obter_codigo_numero_contrato():
    arquivo = 'input/contratos-controle.xlsx'
    df = pd.read_excel(arquivo, sheet_name='contratos')
    cod_num_contrato = {}
    for _idx, row in df[['COD_CONTRATO', 'NUM_CONTRATO']].iterrows():
        cod_num_contrato.update({int(row['COD_CONTRATO']): row['NUM_CONTRATO']})
    return cod_num_contrato

In [ ]:
CONTRATO = obter_codigo_numero_contrato()

In [ ]:
OS = obter_codigo_nome_os()

In [ ]:
dff = pd.DataFrame()
for arquivo in listar_arquivos():
    os_id, contrato_id = obter_os_contrato(arquivo)
    dados = ler_json(arquivo)
    df = pd_json.json_normalize(dados)
    df['os_id'] = os_id
    df['os_nome'] = OS[os_id]
    df['contrato_id'] = contrato_id
    df['contrato_numero'] = CONTRATO[contrato_id]
    dff = dff.append(df)

In [ ]:
dff[(dff['os_id'] == 258) & (dff['valor'] > 0)]